In [1]:
PATH_DATA = 'data/'
FILE = 'votacao_candidato_munzona_2022_BRASIL.csv'

In [2]:
colunas_relevantes = ["NR_TURNO","DS_ELEICAO","TP_ABRANGENCIA","SG_UF","NM_MUNICIPIO","NR_ZONA","DS_CARGO","NR_CANDIDATO","NM_CANDIDATO","NM_URNA_CANDIDATO","DS_SITUACAO_CANDIDATURA","NR_PARTIDO","SG_PARTIDO","NM_PARTIDO","NM_COLIGACAO","DS_COMPOSICAO_COLIGACAO","ST_VOTO_EM_TRANSITO","QT_VOTOS_NOMINAIS","NM_TIPO_DESTINACAO_VOTOS","QT_VOTOS_NOMINAIS_VALIDOS","DS_SIT_TOT_TURNO"]
colunas_relevantes_2 = ["NR_TURNO","SG_UF","NM_MUNICIPIO","NR_CANDIDATO","NM_URNA_CANDIDATO","SG_PARTIDO","ST_VOTO_EM_TRANSITO","QT_VOTOS_NOMINAIS_VALIDOS","DS_SIT_TOT_TURNO","TP_AGREMIACAO","DS_CARGO"]
colunas_relevantes_2.sort()

In [3]:
import time
def create_monitor_file():
    header = "type,function,seconds"
    try:
        with open("monitor.txt", "x") as file:
            file.write(header)
    except FileExistsError:
        pass
def execution_time_logger(name,tipo):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = round(end_time - start_time,2)

            create_monitor_file()
            with open("monitor.txt", "a") as file:
                file.write(f'\n{tipo},{name},{execution_time}')
            return result
        return wrapper
    return decorator


In [4]:
from pyspark.sql import SparkSession
spark = None
@execution_time_logger(name="leitura",tipo='pyspark')
def leitura():
    # Create SparkSession
    global spark
    spark = SparkSession.builder \
        .appName("csv to parquet Conversion") \
        .getOrCreate()

    df = spark.read \
        .option("delimiter", ";") \
        .option("decimal", ",") \
        .option("encoding", "ISO-8859-1") \
        .option("header", True) \
        .csv(PATH_DATA + FILE) \
        .select(colunas_relevantes_2)
    return df

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.functions import when
@execution_time_logger(name="transformacao",tipo='pyspark')
def transforma(df):
    df1 = df.withColumn('SG_UF', F.when(
        df['SG_UF'] == 'ZZ', 'EXTERIOR').otherwise(df['SG_UF']))

    df1 = df1.withColumn("SG_UF_DETALHAMENTO",
                        when(df1["SG_UF"] == "EXTERIOR", df1["SG_UF"])
                        .otherwise(F.concat(F.lit('BRASIL - '), df1['SG_UF'])))

    df1 = df1.withColumn('NM_MUNICIPIO', F.when(
        df1['SG_UF'] == 'EXTERIOR', 'EXTERIOR').otherwise(df1['NM_MUNICIPIO']))
    
    colunas_agrupar = df1.columns
    colunas_agrupar.remove('QT_VOTOS_NOMINAIS_VALIDOS')
    df1 = df1.groupby(colunas_agrupar).sum()

    #Ajuste de Variáveis
    dicionario = {'N': 'NÃO','S':'SIM'}
    df1 = df1.replace(dicionario,subset=['ST_VOTO_EM_TRANSITO'])

    df1 = df1.withColumn("NR_TURNO",df1.NR_TURNO.cast("string"))
    dicionario = {'1': 'PRIMEIRO','2':'SEGUNDO'}
    df1 = df1.replace(dicionario,subset=['NR_TURNO'])
    
    dicionario = {'ELEITO POR QP': 'ELEITO POR QUOCIENTE PARTIDÁRIO'}
    df1 = df1.replace(dicionario,subset=['DS_SIT_TOT_TURNO'])

    df1 = df1.withColumn("DS_CARGO", F.upper(df1.DS_CARGO))
    df1 = df1.withColumn("TP_AGREMIACAO", F.upper(df1.TP_AGREMIACAO))
    return df1
@execution_time_logger(name="parquet",tipo='pyspark')
def parquet(df):
    df.write.mode("overwrite").parquet("data/servico/py_spark_eleicoes_servico") 


In [14]:
import os, shutil
for i in range(10):
    df = leitura()
    df1 = transforma(df) 
    del df
    parquet(df1)
    del df1
    os.rmdir("data/servico/py_spark_eleicoes_servico")

24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/08 14:39:27 WARN RowBasedKeyValueBatch: Calling spill() on

OSError: [Errno 39] Directory not empty: 'data/servico/py_spark_eleicoes_servico'